In [166]:
from glob import glob
import regex as re
import pandas as pd
import cmpd_web
def argsort(seq, key=lambda x: x):
    x = sorted([(v, i) for (i, v) in enumerate(seq)], key=lambda x: key(x[0]))
    return [i for (v, i) in x]

In [167]:
cd elm-src/

[Errno 2] No such file or directory: 'elm-src/'
/Users/feldman/packages/CMPD/elm-src


In [168]:
srcs = glob('*.elm') + glob('*/*.elm')

In [169]:
types = []
for src in srcs:
    txt = open(src, 'r').read()
    tmp = get_types(txt)
    if tmp:
        types += [(src, tmp)]

In [170]:
arr = []
for src, defns in types:
    for defn in defns:
        arr +=[ [src, defn[0], defn[1]]]

df = pd.DataFrame(arr, columns=['elm file', 'type alias', 'definition'])

df['python'] = ['cmpd_web.%s' % alias if alias in dir(cmpd_web) else '??' 
                    for alias in df['type alias']]
df = df[['elm file', 'type alias', 'python', 'definition']]
df = df.set_index('type alias')
ix = argsort(df.index, lambda x: (x=='Model', x))
df = df.iloc[ix]
df.to_clipboard()

In [ ]:
def initialize_map(map_name):
    
    def f(emit):

        print 'initializing map: %s' % map_name

        map_image = maps[map_name]['image']
        places = maps[map_name]['places']
        places = [Place(*p) for p in places]
        player_vocab = load_vocab(cmpd_web.maps[map_name]['vocab'])

        new_vocab = []
        for pos, words in player_vocab:
            words = list(words)
            random.shuffle(words)
            new_vocab += [(pos, words[:12])]
        player_vocab = new_vocab

        map_src = url_for('static', filename=map_image)

        enemies = {}
        for enemy in stable:
            enemies[enemy] = dict(stable[enemy])
            enemies[enemy]['image'] = url_for('static', 
                                                filename=enemies[enemy]['image'])


        player = Player(player_vocab, None, capacity=6)
        player.model['image'] = url_for('static', filename='images/back.png')
        player.model['name'] = 'player'
        player.model['health'] = 0.5
        GM = GameMaster(places, enemies, player, map_src)
        GM.initialize(emit)
        session['GM'] = GM

In [90]:
def despace(s):
    while '  ' in s:
        s = s.replace('  ' , ' ')
    return s.strip()

def get_types(txt):
    defns = []
    lines = txt.splitlines()
    for i, line in enumerate(lines):
        if line.startswith('type alias'):
            type_alias = line.split()[2]
            defn = ''
            for line_ in lines[i+1:]:
                if line_ == '':
                    break
                defn += ' ' + despace(line_)
            defn = defn.replace(' ,', ',').strip()
            defns += [(type_alias, defn)]
    return defns

get_types(txt)

[('AttributeList', 'List (Html.Attribute Msg)'),
 ('Model',
  '{ value : Float, duration : Time.Time, clock : Time.Time, animation : Animation, boxAttributes : AttributeList, sliderAttributes : AttributeList, horizontal : Bool, sliderClasses : List String, boxClasses : List String }')]

### twine

transform harlowe passages into 

function that takes current passage and story variables and returns
- node appropriate for consumption by display
    - list of linked passages is included

### need rules for different passage tags

- **map**
    - `$mapImage` becomes backdrop, defaults to oval office
    - `$places` becomes places, defaults to 'abcd...' in a circle
        - alternative: highlight objects or outlines with same-size transparent png

- **encounter**
    - enemy determined by text with `<enemy|` tag
        - footer checks this against existing enemies, should generate with cmpd.web
        - **rather than the following crap, define the enemies entirely in python for now**
    - image with `<enemyImage|` nametag can override image in stable
        - extract URL
    - override other properties as needed
        - `<enemyClass|`
        - `<**kwargs|` to enemyClass init

- **message**
    - separate passage, but can overlay on previous if `overlay` tag present
    - links are choices, can re-use map-menu Component or rewrite to map keys to class="select"
    - fancy stuff later
    

- **all passages**    
    - transitions ( `HarloweLink` ) made with `(link: )(goto: )` syntax can contain `(set: )` macros
    - after conditions are applied links with <instant| tag trigger an immediate transition from the GameMaster

- **walkabout**
    - why not

In [1]:
from glob import glob
import re
from collections import namedtuple
import sys
sys.path.append('/Users/feldman/packages/harlowe-processor/')

In [79]:
passage_filter = 'Dummy office', 'Challenge ctenophora', 'Images'

In [88]:
dummy_office = [m for m in maps if m.name == 'Dummy office'][0]
challenge_c =  encounters['Challenge ctenophora']

Gamemaster sets up the permitted transitions
transition is called, don't use previous state if possible
Gamemaster has method to update elm based on node type
    - send encounter
    - send map
    - send message

<module 'external.harlowe_extra' from 'external/harlowe_extra.py'>

In [266]:
reload(external.harlowe_extra)
from external.harlowe_extra import html_to_nodes
reload(cmpd_web)
from cmpd_web import HarloweLocal
vocab = cmpd_web.load_vocab('derp')
grammar = None
player = cmpd_web.Player(vocab, grammar)

In [268]:
maps, encounters = html_to_nodes(html)

map with 4 places in Gallery of Heroes
map with 7 places in Oval Office
map with 1 places in Dummy office
generalAbrams found in Fight general
ctenophora found in Fight ctenophora
ctenophora found in Challenge ctenophora


In [269]:
encounters

{'Challenge ctenophora': 'ctenophora',
 'Fight ctenophora': 'ctenophora',
 'Fight general': 'generalAbrams'}

In [327]:
reload(cmpd_web)
from cmpd_web import GameMaster, Player, LocalGameMaster
vocab = cmpd_web.load_vocab('derp')
player = Player(vocab, grammar)
GM = LocalGameMaster([], 'Dummy office', player)
GM.load_html(html)

GM.initialize()

map with 4 places in Gallery of Heroes
map with 7 places in Oval Office
map with 1 places in Dummy office
generalAbrams found in Fight general
ctenophora found in Fight ctenophora
ctenophora found in Challenge ctenophora
image: https://s4.postimg.io/3otw0adgd/ovaloffice.png
places: [Place(x=0.24, y=0.77, key='c', label='Challenge ctenophora')]
Current map: Dummy office
Go to
Challenge ctenophora



In [313]:
GM.transition('Challenge ctenophora')

encounter with {'image': 'images/ctenophora.png', 'health': 1, 'name': 'ctenophora'}
Wordbank: Lazy, Stupid, Insecure, Idiotic, ...
Enemy: Enemy(name='ctenophora', image='images/ctenophora.png', cls=<class 'cmpd_web.Opponent'>, vocab='more')
Enter text to insult, enter nothing to quit.
vapid
Wordbank: Douche, Ass, Turd, Butt, ...
butt
Wordbank: Pilot, Captain, Pirate, Knob, ...
pilot
#################################################
# [--------------------------                 ] #
# Stupid Butt Pilot (0.10)                      #
#                               damaged fascist #
#################################################
Wordbank: Lazy, Stupid, Insecure, Idiotic, ...



In [334]:
maps

{'Dummy office': Map(name='Dummy office', image='https://s4.postimg.io/3otw0adgd/ovaloffice.png', places=[Place(x=0.24, y=0.77, key='c', label='Challenge ctenophora')]),
 'Gallery of Heroes': Map(name='Gallery of Heroes', image='http://online.rapidresizer.com/patternizeimg.php?builtin=logo&id=4cc46cdf56dbdec3f2cde2e1b78f6c5d&bw=1&edge=1&threshold=64&w=', places=[Place(x=0.24, y=0.77, key='n', label=u'RMN'), Place(x=0.06, y=0.46, key='e', label=u'Ehrlichmann'), Place(x=0.27, y=0.42, key='c', label=u'Colson'), Place(x=0.72, y=0.27, key='h', label=u'Haldeman')]),
 'Oval Office': Map(name='Oval Office', image='https://s4.postimg.io/3otw0adgd/ovaloffice.png', places=[Place(x=0.24, y=0.77, key='a', label='War room'), Place(x=0.06, y=0.46, key='v', label="President's phone"), Place(x=0.27, y=0.42, key='i', label='Presidential rocking chair'), Place(x=0.72, y=0.27, key='g', label='Cowboy statue'), Place(x=0.61, y=0.22, key='s', label='Flag of Nixon'), Place(x=0.5, y=0.9, key='d', label='RMN'),

In [323]:
p = passages['Dummy office']
p.contents

"An office for losers.\n=><=\n[]<map|\n(set: $mapImage to $ovalOfficeImage)\n<=\n\n[[Challenge the beast->Challenge ctenophora]]\n\n\n(set: $places to \n\t(a:\n\t\t(a: 0.24, 0.77, 'c'),\n\t\t(a: 0.06, 0.46, 'v'),\n\t\t(a: 0.27, 0.42, 'i'),\n\t\t(a: 0.72, 0.27, 'g'),\n\t\t(a: 0.61, 0.22, 's'),\n\t\t(a: 0.50, 0.90, 'd'),\n\t\t\n\t)\n)\t\t \n\t\t\t "

### modify namedtuple to transmit as dict...

In [152]:
reload(cmpd_web)
from cmpd_web import stable

import external.harlowe_extra
reload(external.harlowe_extra)
from external.harlowe_extra import (get_set_macros, 
                                    parse_harlowe_html, 
                                    get_set_args,
                                    get_image_urls,
                                    get_named_hooks,
                                    find_map,
                                    find_encounter,
                                    find_links)

htmls = glob('/Users/feldman/Downloads/test*.html')
html = htmls[0]
print html

a, b, passages = parse_harlowe_html(html)

# find global image variables
image_passages = [p for n,p in passages.items() if 'Images' in n]
image_urls = {}
[image_urls.update(get_image_urls(p)) for p in image_passages]


def build_graph(html):
    a, b, passages = parse_harlowe_html(html)
    
    # find global image variables
    image_passages = [p for n,p in passages.items() if 'Images' in n]
    image_urls = {}
    [image_urls.update(get_image_urls(p)) for p in image_passages]
    
    filter_out = 'header', 'footer', 'startup'
    remaining_passages = [p for p in passages.values()
                             if not any(f in p.tags for f in filter_out)]
    
    # find maps
    maps = [find_map(p, image_urls) for p in remaining_passages]
    maps = {m.name: m for m in maps if m}
    
    # find encounters
    remaining_passages = [p for p in passages.values() 
                              if p.name not in maps]
    encounters = {p.name: find_encounter(p) for p in remaining_passages}
    encounters = {k: v for k,v in encounters.items() if v}
    
    # check graph integrity
    # missing links, ...
    
    return maps, encounters


maps, encounters = build_graph(html)
maps, encounters

/Users/feldman/Downloads/test.html
map with 4 places in Gallery of Heroes
no <map| nametag found in RMN
no <map| nametag found in Ehrlichmann
no <map| nametag found in Colson
no <map| nametag found in Haldeman
no <map| nametag found in Hall of Champions
map with 7 places in Oval Office
no <map| nametag found in President's phone
no <map| nametag found in Cowboy statue
no <map| nametag found in Presidential rocking chair
no <map| nametag found in Flag of Nixon
no <map| nametag found in Fight general
no <map| nametag found in War room
no <map| nametag found in Fight ctenophora
no <map| nametag found in Spiro Agnew
map with 1 places in Dummy office
no <map| nametag found in Challenge ctenophora
no <enemy| nametag found in RMN
no <enemy| nametag found in Images
no <enemy| nametag found in Ehrlichmann
no <enemy| nametag found in Colson
no <enemy| nametag found in Haldeman
no <enemy| nametag found in Hall of Champions
no <enemy| nametag found in President's phone
no <enemy| nametag found in 

({'Dummy office': Map(name='Dummy office', image='https://s4.postimg.io/3otw0adgd/ovaloffice.png', places=[Place(x=0.24, y=0.77, key='c', label='Challenge ctenophora')]),
  'Gallery of Heroes': Map(name='Gallery of Heroes', image='http://online.rapidresizer.com/patternizeimg.php?builtin=logo&id=4cc46cdf56dbdec3f2cde2e1b78f6c5d&bw=1&edge=1&threshold=64&w=', places=[Place(x=0.24, y=0.77, key='n', label=u'RMN'), Place(x=0.06, y=0.46, key='e', label=u'Ehrlichmann'), Place(x=0.27, y=0.42, key='c', label=u'Colson'), Place(x=0.72, y=0.27, key='h', label=u'Haldeman')]),
  'Oval Office': Map(name='Oval Office', image='https://s4.postimg.io/3otw0adgd/ovaloffice.png', places=[Place(x=0.24, y=0.77, key='a', label='War room'), Place(x=0.06, y=0.46, key='v', label="President's phone"), Place(x=0.27, y=0.42, key='i', label='Presidential rocking chair'), Place(x=0.72, y=0.27, key='g', label='Cowboy statue'), Place(x=0.61, y=0.22, key='s', label='Flag of Nixon'), Place(x=0.5, y=0.9, key='d', label='RMN

### parse conditions

In [ ]:
for i, pc in enumerate(p.parsed_contents):
    arr = []
    if isinstance(pc, HarloweMacro):
        if pc.canonical_name == 'if':
            # attach condition to following hook
            

In [128]:
def transform_macro(hm):
    if hm.canonical_name == 'if':
        code = ''
        for c in hm.code:
            if isinstance(c, str):
                code += c
            if isinstance(c, harlowe.HarloweVariable):
                code += 'hv.' + hv.name
        return code
    

### room map

In [13]:
for name, passage in passages.items():
    passage.parse_contents()
    for c in passage.parsed_contents:
        if isinstance(c, harlowe.HarloweLink):
            passage.destinations |= {c.passage_name[0]}
            passages[c.passage_name[0]].parents |= {name}
            
        

In [14]:
for passage in passages.values():
    print passage.name
    print passage.parents,'->', passage.destinations

Gallery of Heroes
set([]) -> set([u'Colson', u'Ehrlichmann', u'RMN', u'Haldeman'])
RMN
set(['Gallery of Heroes']) -> set([])
global
set([]) -> set([])
Ehrlichmann
set(['Gallery of Heroes']) -> set([])
Colson
set(['Gallery of Heroes']) -> set([])
Haldeman
set(['Gallery of Heroes']) -> set([])
Hall of Champions
set([]) -> set([])
